In [ ]:
# %%  ⏩  Improved version – drop into Jupyter
import warnings, os, sys
warnings.filterwarnings("ignore")
sys.path.append("src")              # allow local imports

import torch
from loguru import logger
from hydra import compose, initialize_config_dir
from omegaconf import DictConfig
from pathlib import Path

# ------------------------------------------------------------------
# 1) Load config
# ------------------------------------------------------------------
config_path = Path.cwd()            # folder that contains config.yaml
with initialize_config_dir(config_dir=str(config_path), version_base=None):
    cfg: DictConfig = compose(config_name="config")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.add(cfg.paths.log_dir + "/run.log", rotation="10 MB")

# ------------------------------------------------------------------
# 2) Download & engineer features
# ------------------------------------------------------------------
from src.data import download_price_news
raw_df = download_price_news(cfg)

from src.features import add_indicators, build_dataset
df = add_indicators(raw_df, cfg.features.indicators)

# ------------------------------------------------------------------
# 3) Walk-forward split (purged)
# ------------------------------------------------------------------
from src.data import walk_forward_split
train_df, val_df, test_df = walk_forward_split(df, train_pct=0.7,
                                               val_pct=0.15, purge=cfg.features.lookback)

train_ds, val_ds, test_ds, scaler = build_dataset(train_df, val_df, test_df, cfg.features.lookback)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=cfg.training.batch_size, shuffle=True)
val_loader   = torch.utils.data.DataLoader(val_ds,   batch_size=cfg.training.batch_size)
test_loader  = torch.utils.data.DataLoader(test_ds,  batch_size=cfg.training.batch_size)

# ------------------------------------------------------------------
# 4) Model & training
# ------------------------------------------------------------------
from src.model import StockNet, EarlyStopper
from src.train import train_one_epoch, evaluate

num_price_feat = train_ds[0][0].shape[-1]
model = StockNet(num_price_feat).to(device)
optimizer = torch.optim.Adam(model.parameters(),
                             lr=cfg.training.lr,
                             weight_decay=cfg.training.weight_decay)
criterion = torch.nn.CrossEntropyLoss()

Path(cfg.paths.model_dir).mkdir(exist_ok=True, parents=True)
best_path = Path(cfg.paths.model_dir) / "best.pt"
stopper   = EarlyStopper(patience=cfg.training.patience, path=best_path)

for epoch in range(1, cfg.training.max_epochs + 1):
    tr_loss, tr_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc, _, _ = evaluate(model, val_loader, criterion, device)
    logger.info(f"Epoch {epoch:03d} | train {tr_acc:.3f} | val {val_acc:.3f}")
    if stopper(val_loss, model):
        logger.info("Early stopping")
        break

# ------------------------------------------------------------------
# 5) Test evaluation
# ------------------------------------------------------------------
model.load_state_dict(torch.load(best_path))
test_loss, test_acc, preds, labels = evaluate(model, test_loader, criterion, device)
logger.success(f"Test accuracy: {test_acc:.3%}")

# ------------------------------------------------------------------
# 6) Quick inference demo
# ------------------------------------------------------------------
from src.predict import StockPredictor
predictor = StockPredictor(best_path, scaler, cfg.features.lookback)

tail = df.tail(cfg.features.lookback + 1).iloc[:-1]
direction, prob = predictor.predict_one(tail, news="Fed cuts rates")
print(f"Prediction: {direction} @ {prob:.2%}")